In [99]:
# Packages
import pandas as pd
import datetime as dtt
from os import getcwd

path_data = f"{getcwd()}/../data/raw/"

# Import CRSP:
#   PERMNO is security identifier for CRSP, SHRCD is security type,
#   SICCD is SIC industry categorization. NCUSIP is historical security identifier.
#   COMNAM is company name, PRIMEXCH is primary exchange, CUSIP is current security identifier
#   PRC is price, VOL is volume, SHROUT is shares outstanding, vwretd and ewretd are value and equally weighted market return averages.
#
# you will need date and NCUSIP to identify mainly these companies. May use Companyname and or Ticker with XBRL if there is no/partial CUSIP identifier
df_crsp = pd.read_csv(path_data+'crsp_monthly.csv')
df_crsp['date'] = pd.to_datetime(df_crsp['date'])

# Import IBES
df_ibes = pd.read_csv(path_data+'ibes_annual.csv')
df_ibes['date'] = pd.to_datetime(df_ibes['ANNDATS'])


# YOU SHOULD MERGE IT ON XBRL data, based on the process described in Table 1
# but here is an example how to merge them together:

# Select needed columns
df_crsp_m = df_crsp[['date','NCUSIP','COMNAM','PRC','SHROUT','vwretd','ewretd']]
df_crsp_m.shape
df_ibes_m = df_ibes[['CUSIP','CNAME','date','VALUE']]
df_ibes_m.shape

# Merge: inner (both has to be in data, note you have to modify it to be in XBRL, and it is not the same stuff necessarily)
df = pd.merge(df_ibes_m, df_crsp_m, how='inner', left_on=['date', 'CUSIP'], right_on=['date','NCUSIP'])

# All obs from CRPS and IBES
print(df.shape)

# Drop duplicates
df = df.drop_duplicates(subset=['date','NCUSIP'])
df.shape

# Available stock price from CRSP (non NA)
df_total = df.dropna(subset=['PRC'])
df_total.shape

##
# NOTES: a) you need to carefully investigate and remove the duplicates form CRSP and IBES and XBRL individually! Check the sample sizes! E.g. here you will end up with way less observations

/var/folders/93/vcxdh9f57dq1xzncrcs6kl_40000gn/T/ipykernel_95542/149541153.py:15: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crsp = pd.read_csv(path_data+'crsp_monthly.csv')


(161193, 10)


(3458, 10)

In [100]:
df_ibes.shape

(396741, 10)

In [101]:
df_crsp.shape

(1176721, 14)

In [102]:
df_total.tail(100)

,CUSIP,CNAME,date,VALUE,NCUSIP,COMNAM,PRC,SHROUT,vwretd,ewretd
160374,92939U10,WEC ENERGY GROUP,2018-01-31,3.14,92939U10,W E C ENERGY GROUP INC,64.30,315576.0,0.050638,0.029441
160379,92931L20,WPCS INTL INC,2011-07-29,NaN,92931L20,W P C S INTERNATIONAL INC,2.99,6955.0,-0.022550,-0.021813
160384,05474810,AYRO,2021-03-31,-0.72,05474810,AYRO INC,6.48,35213.0,0.030573,0.012449
160389,96090830,WESTPORT INNOV,2012-02-29,-1.26,96090830,WESTPORT INNOVATIONS INC,40.47,54846.0,0.041253,0.036494
160394,45822P10,INTEGRYS ENERGY,2013-02-28,3.26,45822P10,INTEGRYS ENERGY GROUP INC,56.57,78288.0,0.008284,0.003491
...,...,...,...,...,...,...,...,...,...,...
161165,N2094410,CNH INDUSTRIAL,2015-01-30,3.63,N2094410,C N H INDUSTRIAL N V,7.65,1354194.0,-0.027207,-0.019073
161168,36268W10,GAIN CAPITAL HOL,2012-02-29,NaN,36268W10,GAIN CAPITAL HOLDINGS INC,5.24,34212.0,0.041253,0.036494
161183,47231910,JEFFERIES GROUP,2010-05-28,NaN,47231910,JEFFERIES GROUP INC NEW,23.33,171690.0,-0.079267,-0.076929
161186,63370710,NATIONAL BANK HL,2015-01-30,20.43,63370710,NATIONAL BANK HOLDINGS CORP,18.45,38922.0,-0.027207,-0.019073


In [103]:
df[df["CNAME"] == "WSB HOLDINGS INC"].head(100)

,CUSIP,CNAME,date,VALUE,NCUSIP,COMNAM,PRC,SHROUT,vwretd,ewretd
160419,92934C10,WSB HOLDINGS INC,2011-01-31,NaN,92934C10,W S B HOLDINGS INC,2.95,7925.0,0.019190,0.014412
160424,92934C10,WSB HOLDINGS INC,2012-01-31,NaN,92934C10,W S B HOLDINGS INC,2.80,7995.0,0.054140,0.085342
160429,92934C10,WSB HOLDINGS INC,2013-01-31,NaN,92934C10,W S B HOLDINGS INC,6.06,8017.0,0.054189,0.063509


In [104]:
df["date"].sort_values().tail(100)

129308   2022-03-31
115216   2022-03-31
118748   2022-03-31
158263   2022-03-31
138436   2022-03-31
            ...    
141811   2022-11-30
115176   2022-11-30
157609   2022-11-30
32211    2022-11-30
115141   2022-11-30
Name: date, Length: 100, dtype: datetime64[ns]

In [105]:
df_ibes.head(100)

,TICKER,CUSIP,OFTIC,CNAME,MEASURE,PDICITY,ANNDATS,VALUE,USFIRM,date
0,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
1,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
2,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2015-01-30,1.21,1,2015-01-30
3,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2016-01-26,1.02,1,2016-01-26
4,0001,NaN,NaN,NaN,EPS,ANN,2014-01-23,NaN,1,2014-01-23
...,...,...,...,...,...,...,...,...,...,...
95,001J,49926D10,KN,KNOWLES,EPS,ANN,2022-02-09,1.53,1,2022-02-09
96,001J,49926D10,KN,KNOWLES,EPS,ANN,2023-02-09,1.26,1,2023-02-09
97,001K,NaN,NaN,NaN,EPS,ANN,2014-02-19,NaN,1,2014-02-19
98,001K,NaN,NaN,NaN,EPS,ANN,2014-03-26,0.02,1,2014-03-26


In [106]:
df_ibes.head()

,TICKER,CUSIP,OFTIC,CNAME,MEASURE,PDICITY,ANNDATS,VALUE,USFIRM,date
0,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
1,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
2,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2015-01-30,1.21,1,2015-01-30
3,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2016-01-26,1.02,1,2016-01-26
4,0001,NaN,NaN,NaN,EPS,ANN,2014-01-23,NaN,1,2014-01-23


In [107]:
df_ibes["ANNDATS"].head(100)

0     2014-02-14
1     2014-02-14
2     2015-01-30
3     2016-01-26
4     2014-01-23
         ...    
95    2022-02-09
96    2023-02-09
97    2014-02-19
98    2014-03-26
99    2015-03-04
Name: ANNDATS, Length: 100, dtype: object

In [108]:
df.head()

,CUSIP,CNAME,date,VALUE,NCUSIP,COMNAM,PRC,SHROUT,vwretd,ewretd
0,87482X10,TALMER BANCORP,2015-01-30,1.21,87482X10,TALMER BANCORP INC,13.5200,70532.0,-0.027207,-0.019073
5,26878510,EP ENGR CORP,2018-02-28,-0.39,26878510,E P ENERGY CORP,1.5100,254923.0,-0.039481,-0.040026
10,NaN,NaN,2014-03-31,-403.65,NaN,NaN,NaN,NaN,0.004544,0.001028
5180,15117E10,CELLADON,2015-03-31,-27.30,15117E10,CELLADON CORP,18.9400,23828.0,-0.010410,-0.005112
5185,81776310,SESEN BIO,2022-02-28,0.00,81776310,SESEN BIO INC,0.6343,199464.0,-0.021838,-0.007412


In [109]:
df["year"] = df["date"].dt.year
df['month'] = df['date'].dt.month

In [110]:
df.head()

,CUSIP,CNAME,date,VALUE,NCUSIP,COMNAM,PRC,SHROUT,vwretd,ewretd,year,month
0,87482X10,TALMER BANCORP,2015-01-30,1.21,87482X10,TALMER BANCORP INC,13.5200,70532.0,-0.027207,-0.019073,2015,1
5,26878510,EP ENGR CORP,2018-02-28,-0.39,26878510,E P ENERGY CORP,1.5100,254923.0,-0.039481,-0.040026,2018,2
10,NaN,NaN,2014-03-31,-403.65,NaN,NaN,NaN,NaN,0.004544,0.001028,2014,3
5180,15117E10,CELLADON,2015-03-31,-27.30,15117E10,CELLADON CORP,18.9400,23828.0,-0.010410,-0.005112,2015,3
5185,81776310,SESEN BIO,2022-02-28,0.00,81776310,SESEN BIO INC,0.6343,199464.0,-0.021838,-0.007412,2022,2


In [111]:
df.head()

,CUSIP,CNAME,date,VALUE,NCUSIP,COMNAM,PRC,SHROUT,vwretd,ewretd,year,month
0,87482X10,TALMER BANCORP,2015-01-30,1.21,87482X10,TALMER BANCORP INC,13.5200,70532.0,-0.027207,-0.019073,2015,1
5,26878510,EP ENGR CORP,2018-02-28,-0.39,26878510,E P ENERGY CORP,1.5100,254923.0,-0.039481,-0.040026,2018,2
10,NaN,NaN,2014-03-31,-403.65,NaN,NaN,NaN,NaN,0.004544,0.001028,2014,3
5180,15117E10,CELLADON,2015-03-31,-27.30,15117E10,CELLADON CORP,18.9400,23828.0,-0.010410,-0.005112,2015,3
5185,81776310,SESEN BIO,2022-02-28,0.00,81776310,SESEN BIO INC,0.6343,199464.0,-0.021838,-0.007412,2022,2


In [112]:
df = df.dropna(subset=["CUSIP", "year", "month"])
df.shape

(3546, 12)

In [88]:
df.to_csv(f"{path_data}/crsp_ibes.csv", index=False)

In [114]:
df_ibes.head(100)

,TICKER,CUSIP,OFTIC,CNAME,MEASURE,PDICITY,ANNDATS,VALUE,USFIRM,date
0,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
1,0000,NaN,NaN,NaN,EPS,ANN,2014-02-14,NaN,1,2014-02-14
2,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2015-01-30,1.21,1,2015-01-30
3,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2016-01-26,1.02,1,2016-01-26
4,0001,NaN,NaN,NaN,EPS,ANN,2014-01-23,NaN,1,2014-01-23
...,...,...,...,...,...,...,...,...,...,...
95,001J,49926D10,KN,KNOWLES,EPS,ANN,2022-02-09,1.53,1,2022-02-09
96,001J,49926D10,KN,KNOWLES,EPS,ANN,2023-02-09,1.26,1,2023-02-09
97,001K,NaN,NaN,NaN,EPS,ANN,2014-02-19,NaN,1,2014-02-19
98,001K,NaN,NaN,NaN,EPS,ANN,2014-03-26,0.02,1,2014-03-26


In [116]:
(df_ibes["ANNDATS"] == df_ibes["date"]).unique()

array([ True])

In [117]:
df_ibes = df_ibes.dropna(subset=["CUSIP", "VALUE", "date"])
df_ibes.shape

(211643, 10)

In [118]:
df_ibes.head(100)

,TICKER,CUSIP,OFTIC,CNAME,MEASURE,PDICITY,ANNDATS,VALUE,USFIRM,date
2,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2015-01-30,1.21,1,2015-01-30
3,0000,87482X10,TLMR,TALMER BANCORP,EPS,ANN,2016-01-26,1.02,1,2016-01-26
6,0001,26878510,EPE,EP ENGR CORP,EPS,ANN,2015-02-18,0.86,1,2015-02-18
7,0001,26878510,EPE,EP ENGR CORP,EPS,ANN,2016-02-18,0.78,1,2016-02-18
8,0001,26878510,EPE,EP ENGR CORP,EPS,ANN,2017-03-01,0.62,1,2017-03-01
...,...,...,...,...,...,...,...,...,...,...
128,001M,45780L10,INGN,INOGEN,EPS,ANN,2023-02-23,-1.40,1,2023-02-23
130,001N,22265J10,COUP,COUPONS.COM,EPS,ANN,2015-02-09,0.09,1,2015-02-09
131,001N,74911910,QUOT,QUOTIENT TECH,EPS,ANN,2016-02-11,0.10,1,2016-02-11
132,001N,74911910,QUOT,QUOTIENT TECH,EPS,ANN,2017-02-09,0.12,1,2017-02-09


In [133]:
df_ibes["PDICITY"].unique()

array(['ANN'], dtype=object)

In [119]:
df_ibes = df_ibes.drop_duplicates(subset=["date", "CUSIP"])

In [120]:
df_ibes.shape

(63570, 10)

In [121]:
df_ibes["MEASURE"].unique()

array(['EPS', 'BPS', 'GPS', 'ENT', 'EBS'], dtype=object)

In [122]:
df_ibes[df_ibes["MEASURE"] == "EPS"].shape

(62313, 10)

In [123]:
df_ibes = df_ibes[df_ibes["MEASURE"] == "EPS"]

In [138]:
df_ibes = df_ibes.drop(["ANNDATS", "PDICITY", "MEASURE"], axis=1)

KeyError: "['ANNDATS', 'PDICITY', 'MEASURE'] not found in axis"

In [142]:
df_ibes['year'] = df_ibes['date'].dt.year
df_ibes['month'] = df_ibes['date'].dt.month

In [143]:
df_ibes.to_csv(f"{path_data}/filtered_ibes.csv", index=False)

In [98]:
df_ibes["CNAME"].unique().shape

(12816,)

In [71]:
correct_set = set(list(range(2014, 2021)))
cache = {}
def see_full_companies(row):
    name = row["CNAME"]
    if name not in cache:
        unique_dates_for_company = df_ibes[df_ibes["CNAME"] == name]["date"].dt.year.unique()
        cache[name] = correct_set.intersection(unique_dates_for_company) == correct_set
    return cache[name]
m = df_ibes.apply(see_full_companies, axis=1)
m.shape

(76181,)

In [72]:
df_ibes.to_csv(f"{path_data}/ibes_filtered.csv")

In [73]:
df_ibes.shape

(76181, 10)